# Correlating Returns

In [2]:
import alpaca_trade_api as tradeapi
import pandas as pd
from newsapi.newsapi_client import NewsApiClient
from datetime import date, datetime, timedelta
import os
#from dotenv import load_dotenv
from nltk.sentiment.vader import SentimentIntensityAnalyzer

## Load API Keys from Environment Variables

In [4]:
# Load .env enviroment variables
#load_dotenv()

# Set News API Key
newsapi = NewsApiClient(api_key=os.environ["NEWS_API_KEY"])

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

api = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version='v2')

## Get AAPL Returns for Past Month

In [5]:
# Set the ticker
ticker = "AAPL"

# Set timeframe to '1D'
timeframe = '1D'

# Get current date and the date from one month ago
current_date = date.today()
past_date = date.today() - timedelta(weeks=4)

# Get 4 weeks worth of historical data for AAPL
df = api.get_barset(
    ticker,
    timeframe,
    limit=None,
    start=current_date,
    end=past_date,
    after=None,
    until=None,
).df

df.head()

AAPL                                     
                             open      high      low   close    volume
time                                                                  
2020-01-06 00:00:00-05:00  293.79  299.9600  292.750  299.78  27487207
2020-01-07 00:00:00-05:00  299.84  300.9000  297.480  298.27  23877417
2020-01-08 00:00:00-05:00  297.16  304.4399  297.156  303.15  29547763
2020-01-09 00:00:00-05:00  307.25  310.4300  306.200  309.71  39240341
2020-01-10 00:00:00-05:00  310.60  312.6700  308.250  310.37  32981826

In [6]:
# Drop Outer Table Level
df = df.droplevel(axis=1, level=0)

# Use the drop function to drop extra columns
df.drop(columns=['open', 'high', 'low', 'volume'], inplace=True)

# Since this is daily data, we can keep only the date (remove the time) component of the data
df.index = df.index.date

df.head()

,close
2020-01-06,299.78
2020-01-07,298.27
2020-01-08,303.15
2020-01-09,309.71
2020-01-10,310.37


In [7]:
# Use the `pct_change` function to calculate daily returns of AAPL
aapl_returns = df.pct_change()
aapl_returns.head()

,close
2020-01-06,NaN
2020-01-07,-0.005037
2020-01-08,0.016361
2020-01-09,0.021639
2020-01-10,0.002131


In [8]:
# Use newsapi client to get most relevant 20 headlines per day in the past month
def get_headlines(keyword):
    all_headlines = []
    all_dates = []
    date = current_date
    print(f"Fetching news about '{keyword}'")
    print("*" * 30)
    while date > past_date:
        print(f"retrieving news from: {date}")
        articles = newsapi.get_everything(
            q=keyword,
            from_param=str(date),
            to=str(date),
            language="en",
            sort_by="relevancy",
            page=1,
        )
        headlines = []
        for i in range(0, len(articles["articles"])):
            headlines.append(articles["articles"][i]["title"])
        all_headlines.append(headlines)
        all_dates.append(date)
        date = date - timedelta(days=1)
    return all_headlines, all_dates



In [9]:
# Get first topic
aapl_headlines, dates = get_headlines("apple")

Fetching news about 'apple'
******************************
retrieving news from: 2020-05-28
retrieving news from: 2020-05-27
retrieving news from: 2020-05-26
retrieving news from: 2020-05-25
retrieving news from: 2020-05-24
retrieving news from: 2020-05-23
retrieving news from: 2020-05-22
retrieving news from: 2020-05-21
retrieving news from: 2020-05-20
retrieving news from: 2020-05-19
retrieving news from: 2020-05-18
retrieving news from: 2020-05-17
retrieving news from: 2020-05-16
retrieving news from: 2020-05-15
retrieving news from: 2020-05-14
retrieving news from: 2020-05-13
retrieving news from: 2020-05-12
retrieving news from: 2020-05-11
retrieving news from: 2020-05-10
retrieving news from: 2020-05-09
retrieving news from: 2020-05-08
retrieving news from: 2020-05-07
retrieving news from: 2020-05-06
retrieving news from: 2020-05-05
retrieving news from: 2020-05-04
retrieving news from: 2020-05-03
retrieving news from: 2020-05-02
retrieving news from: 2020-05-01


In [10]:
# Get second topic
trade_headlines, _ = get_headlines("trade")

Fetching news about 'trade'
******************************
retrieving news from: 2020-05-28
retrieving news from: 2020-05-27
retrieving news from: 2020-05-26
retrieving news from: 2020-05-25
retrieving news from: 2020-05-24
retrieving news from: 2020-05-23
retrieving news from: 2020-05-22
retrieving news from: 2020-05-21
retrieving news from: 2020-05-20
retrieving news from: 2020-05-19
retrieving news from: 2020-05-18
retrieving news from: 2020-05-17
retrieving news from: 2020-05-16
retrieving news from: 2020-05-15
retrieving news from: 2020-05-14
retrieving news from: 2020-05-13
retrieving news from: 2020-05-12
retrieving news from: 2020-05-11
retrieving news from: 2020-05-10
retrieving news from: 2020-05-09
retrieving news from: 2020-05-08
retrieving news from: 2020-05-07
retrieving news from: 2020-05-06
retrieving news from: 2020-05-05
retrieving news from: 2020-05-04
retrieving news from: 2020-05-03
retrieving news from: 2020-05-02
retrieving news from: 2020-05-01


In [17]:
# Get third topic
economy_headlines, _ = get_headlines("economy")

Fetching news about 'economy'
******************************
retrieving news from: 2020-05-28
retrieving news from: 2020-05-27
retrieving news from: 2020-05-26
retrieving news from: 2020-05-25
retrieving news from: 2020-05-24
retrieving news from: 2020-05-23
retrieving news from: 2020-05-22
retrieving news from: 2020-05-21
retrieving news from: 2020-05-20
retrieving news from: 2020-05-19
retrieving news from: 2020-05-18
retrieving news from: 2020-05-17
retrieving news from: 2020-05-16
retrieving news from: 2020-05-15
retrieving news from: 2020-05-14
retrieving news from: 2020-05-13
retrieving news from: 2020-05-12
retrieving news from: 2020-05-11
retrieving news from: 2020-05-10
retrieving news from: 2020-05-09
retrieving news from: 2020-05-08
retrieving news from: 2020-05-07
retrieving news from: 2020-05-06
retrieving news from: 2020-05-05
retrieving news from: 2020-05-04
retrieving news from: 2020-05-03
retrieving news from: 2020-05-02
retrieving news from: 2020-05-01


In [18]:
# Get fourth topic
iphone_headlines, _ = get_headlines("iphone")

Fetching news about 'iphone'
******************************
retrieving news from: 2020-05-28
retrieving news from: 2020-05-27
retrieving news from: 2020-05-26
retrieving news from: 2020-05-25
retrieving news from: 2020-05-24
retrieving news from: 2020-05-23
retrieving news from: 2020-05-22
retrieving news from: 2020-05-21
retrieving news from: 2020-05-20
retrieving news from: 2020-05-19
retrieving news from: 2020-05-18
retrieving news from: 2020-05-17
retrieving news from: 2020-05-16
retrieving news from: 2020-05-15
retrieving news from: 2020-05-14
retrieving news from: 2020-05-13
retrieving news from: 2020-05-12
retrieving news from: 2020-05-11
retrieving news from: 2020-05-10
retrieving news from: 2020-05-09
retrieving news from: 2020-05-08
retrieving news from: 2020-05-07
retrieving news from: 2020-05-06
retrieving news from: 2020-05-05
retrieving news from: 2020-05-04
retrieving news from: 2020-05-03
retrieving news from: 2020-05-02
retrieving news from: 2020-05-01


In [19]:
# Get fifth topic
gold_headlines, _ = get_headlines("gold")

Fetching news about 'gold'
******************************
retrieving news from: 2020-05-28
retrieving news from: 2020-05-27
retrieving news from: 2020-05-26
retrieving news from: 2020-05-25
retrieving news from: 2020-05-24
retrieving news from: 2020-05-23
retrieving news from: 2020-05-22
retrieving news from: 2020-05-21
retrieving news from: 2020-05-20
retrieving news from: 2020-05-19
retrieving news from: 2020-05-18
retrieving news from: 2020-05-17
retrieving news from: 2020-05-16
retrieving news from: 2020-05-15
retrieving news from: 2020-05-14
retrieving news from: 2020-05-13
retrieving news from: 2020-05-12
retrieving news from: 2020-05-11
retrieving news from: 2020-05-10
retrieving news from: 2020-05-09
retrieving news from: 2020-05-08
retrieving news from: 2020-05-07
retrieving news from: 2020-05-06
retrieving news from: 2020-05-05
retrieving news from: 2020-05-04
retrieving news from: 2020-05-03
retrieving news from: 2020-05-02
retrieving news from: 2020-05-01


In [20]:
# Instantiate SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

In [21]:
# Create function that computes average compound sentiment of headlines for each day
def headline_sentiment_summarizer_avg(headlines):
    sentiment = []
    for day in headlines:
        day_score = []
        for h in day:
            if h == None:
                continue
            else:
                day_score.append(sid.polarity_scores(h)["compound"])
        sentiment.append(sum(day_score) / len(day_score))
    return sentiment


In [22]:
# Get averages of each topics sentiment
aapl_avg = headline_sentiment_summarizer_avg(aapl_headlines)
trade_avg = headline_sentiment_summarizer_avg(trade_headlines)
economy_avg = headline_sentiment_summarizer_avg(economy_headlines)
iphone_avg = headline_sentiment_summarizer_avg(iphone_headlines)
gold_avg = headline_sentiment_summarizer_avg(gold_headlines)

In [26]:
# Combtopic_sentiments = pd.DataFrame(
    {
        "aapl_avg": aapl_avg,
        "trade_avg": trade_avg,
        "economy_avg": economy_avg,
        "iphone_avg": iphone_avg,
        "gold_avg": gold_avg,
    }
)


IndentationError: unexpected indent (<ipython-input-26-0dfd5db35d03>, line 2)

In [27]:
# Set the index value of the sentiment average DataFrame to be the series of dates.
topic_sentiments.index = pd.to_datetime(dates)

NameError: name 'topic_sentiments' is not defined

In [6]:
# Merge with AAPL returns


In [7]:
# Correlate the headlines' sentiment to returns
